In [1]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum

# Create the problem
problem = LpProblem("Simple_MIP_Problem", LpMaximize)

# Define decision variables
x = LpVariable("x", lowBound=0, cat="Integer")  # x is an integer variable
y = LpVariable("y", lowBound=0, cat="Continuous")  # y is a continuous variable

# Define the objective function
problem += 3 * x + 5 * y, "Objective"

# Define the constraints
problem += 2 * x + 3 * y <= 12, "Constraint_1"
problem += x + y <= 4, "Constraint_2"

# Solve the problem
problem.solve()

# Print the results
print("Status:", problem.status)
print("Optimal Solution:")
print(f"x = {x.varValue}")
print(f"y = {y.varValue}")
print("Objective value =", problem.objective.value())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/vivekchaudhary/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/tt/3h_jlt8571z664b95jr80v_40000gn/T/4d94a228e5b24fdcaf2ccf46e960fdce-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/tt/3h_jlt8571z664b95jr80v_40000gn/T/4d94a228e5b24fdcaf2ccf46e960fdce-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 7 COLUMNS
At line 16 RHS
At line 19 BOUNDS
At line 21 ENDATA
Problem MODEL has 2 rows, 2 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 20 - 0.00 seconds
Cgl0004I processed model has 2 rows, 2 columns (1 integer (0 of which binary)) and 4 elements
Cbc0012I Integer solution of -20 found by DiveCoefficient after 0 iterations and 0 nodes (0.02 seconds)
Cbc0001I Search completed - best objective -20, took 0 itera

In [2]:
import numpy as np
from scipy.optimize import linprog

def branch_and_cut(c, A, b, integer_indices, bounds):
    """
    Solves a Mixed Integer Linear Programming problem using a simple Branch-and-Cut algorithm.

    Args:
    - c: Coefficients of the objective function.
    - A: Coefficients of the constraints.
    - b: Right-hand side of the constraints.
    - integer_indices: Indices of variables that must be integers.
    - bounds: Variable bounds [(lower, upper), ...]

    Returns:
    - Optimal solution vector and the objective value.
    """
    best_solution = None
    best_value = -np.inf
    nodes = [(A, b, bounds)]  # Stack for Branch-and-Bound nodes

    while nodes:
        # Pop a node from the stack
        current_A, current_b, current_bounds = nodes.pop()

        # Solve the relaxed problem
        res = linprog(c, A_ub=current_A, b_ub=current_b, bounds=current_bounds, method='highs')
        
        if res.success:
            solution = res.x
            value = res.fun

            # Check if all integer variables are satisfied
            if all(np.isclose(solution[i], round(solution[i]), atol=1e-6) for i in integer_indices):
                if value > best_value:
                    best_solution = solution
                    best_value = value
            else:
                # Branch on the first fractional integer variable
                for i in integer_indices:
                    if not np.isclose(solution[i], round(solution[i]), atol=1e-6):
                        lower_bound = np.floor(solution[i])
                        upper_bound = np.ceil(solution[i])

                        # Create new nodes by branching
                        new_bounds1 = current_bounds[:]
                        new_bounds1[i] = (current_bounds[i][0], lower_bound)

                        new_bounds2 = current_bounds[:]
                        new_bounds2[i] = (upper_bound, current_bounds[i][1])

                        nodes.append((current_A, current_b, new_bounds1))
                        nodes.append((current_A, current_b, new_bounds2))
                        break

    return best_solution, -best_value  # Convert to maximization


# Problem Definition
c = [-3, -5]  # Objective coefficients (minimize -z for maximization)
A = [[2, 3], [1, 1]]  # Constraint coefficients
b = [12, 4]  # Right-hand side of the constraints
integer_indices = [0]  # Indices of integer variables (x is integer)
bounds = [(0, None), (0, None)]  # Bounds for x and y

# Solve the problem
solution, optimal_value = branch_and_cut(c, A, b, integer_indices, bounds)

print("Optimal Solution:")
print("x =", solution[0])
print("y =", solution[1])
print("Objective value =", optimal_value)



Optimal Solution:
x = 0.0
y = 4.0
Objective value = 20.0
